# GSMM mixture modelling with synthetic data

This notebook contains the following processing blocks:
- Gaussian mixture model
- Complex hierarchical gaussian filter
- Complex to real conversion
- Observation noise

## Import packages and functions

In [1]:
import Pkg; Pkg.activate("C:/Users/s151781/AppData/Local/Julia-1.3.1/GN/Project.toml")
using Revise
using FFTW
using Compat
using WAV
using DSP
using Base64
using ForneyLab
using LinearAlgebra
using ProgressMeter
using PyPlot
using GaussianMixtures
;

Activating environment at `C:\Users\s151781\AppData\Local\Julia-1.3.1\GN\Project.toml`


In [2]:
include("../extensions/ComplexNormal.jl")
include("../extensions/ComplexHGF.jl")
include("../extensions/ComplexToReal.jl")
include("../functions/auxiliary/workflow.jl") # for some workflow simplifications
;

## Parameters

In [3]:
# data generation parameters
nr_samples = 200                     # number of samples
nr_freqs = 5                         # number of frequencies
nr_clusters = 3                      # number of clusters (needs to be larger than 2 (ForneyLab issue))
fs = 1                               # sampling frequency

Σ_ξ = 1e-4*Ic(nr_freqs)              # covariance matrix of ξ
Σ_meas = 1e-10*Ic(2*nr_freqs)        # covariance matrix of measurement noise (over frequencies (real + imag) in this case)
;

## Generate data

In [4]:
# import sampling functions
import Distributions: Normal, MvNormal, MixtureModel, Dirichlet

# set means of ξ for the different clusters
μ_ξ = vcat([collect(k:k:k*nr_freqs) for k = 1:nr_clusters])

# create arrays over samples
cluster_id = Array{Int64,1}(undef, nr_samples)
ξ_samples = Array{Array{Float64,1},1}(undef, nr_samples)
Xc_samples = Array{Array{Complex{Float64},1},1}(undef, nr_samples)
Xr_samples = Array{Array{Float64,1},1}(undef, nr_samples)
Y_samples = Array{Array{Float64,1},1}(undef, nr_samples)

# generate samples independent from each other
for n = 1:nr_samples
     
    # create arrays over frequencies
    ξ_samples[n] = Array{Float64,1}(undef, nr_freqs)
    Xc_samples[n] = Array{Complex{Float64},1}(undef, nr_freqs)
    Xr_samples[n] = Array{Float64,1}(undef, 2*nr_freqs)
    Y_samples[n] = Array{Float64,1}(undef, 2*nr_freqs)
    
    # randomly pick cluster to get means from 
    cluster_id[n] = rand(collect(1:nr_clusters))
    μ_ξi = μ_ξ[cluster_id[n]]
    
    # calculate samples fro frequencies
    for k = 1:nr_freqs
        
        # generate samples (log-power domain)
        sample_ξ = rand(Normal(μ_ξi[k], sqrt(Σ_ξ[k,k])))
        
        # generate samples (complex frequency coefs)
        sample_Xc = rand(Normal(0, sqrt(0.5*exp(sample_ξ)))) + 1im*rand(Normal(0, sqrt(0.5*exp(sample_ξ))))
        
        # generate samples (real frequency coefs)
        sample_Xr = hcat(real.(sample_Xc), imag.(sample_Xc))
        
        # generate samples (noisy observations)
        sample_Y = [rand(Normal(sample_Xr[1], sqrt(Σ_meas[2*k-1,2*k-1]))), rand(Normal(sample_Xr[2], sqrt(Σ_meas[2*k,2*k])))]

        # save samples
        ξ_samples[n][k] = sample_ξ
        Xc_samples[n][k] = sample_Xc
        Xr_samples[n][k] = sample_Xr[1]
        Xr_samples[n][k+nr_freqs] = sample_Xr[2]
        Y_samples[n][k] = sample_Y[1]
        Y_samples[n][k+nr_freqs] = sample_Y[2]
        
    end
    
end

# create time axis
t = collect(1:nr_samples)/fs
;

## Build factor graph

In [5]:
# create factor graph
fg = FactorGraph()

# create distionary for variables
vars = Dict()

# specify distribution over the selection variables
@RV vars[:π] ~ ForneyLab.Dirichlet(placeholder(:α_π, dims=(nr_clusters,)))

# create mixture components
for k = 1:nr_clusters
    
    # specify distribution over precision matrix
    @RV vars[pad(:w,k)] ~ Wishart(placeholder(pad(:V_w,k), dims=(nr_freqs,nr_freqs)), placeholder(pad(:nu_w,k)))
    
    # specify distribution over mean
    @RV vars[pad(:m,k)] ~ GaussianMeanPrecision(placeholder(pad(:μ_m,k), dims=(nr_freqs,)), vars[pad(:w,k)])
    
end

# create sample-dependent random variables
for k = 1:nr_samples
    
    # specify distribution over selection variable
    @RV vars[pad(:z,k)] ~ Categorical(vars[:π])
    
    # create gaussian mixture model
    @RV vars[pad(:ξ,k)] ~ GaussianMixture(vars[pad(:z,k)], expand([[vars[pad(:m,ki)], vars[pad(:w,ki)]] for ki=1:nr_clusters])...)
    
    # log-power to complex fourier coefficients transform
    @RV vars[pad(:Xc,k)] ~ ComplexHGF(vars[pad(:ξ,k)])

    # complex fourier coefficients to real and imaginary parts concatenated
    @RV vars[pad(:Xr,k)] ~ ComplexToReal(vars[pad(:Xc,k)])
    
    # observation model 
    @RV vars[pad(:Y,k)] ~ GaussianMeanVariance(vars[pad(:Xr,k)], Σ_meas)
    
    # observation
    placeholder(vars[pad(:Y,k)], :Y, index=k, dims=(2*nr_freqs,))
    
end

# draw graph
# ForneyLab.draw(fg)
;

## Generate inference algorithm

In [6]:
# specify ids for the posterior factorization
q_ids = vcat(:Π,
              expand([[pad(:M,k), pad(:W,k)] for k=1:nr_clusters]),
              :Z, :Xc, :Ξ)

# specify posterior factorization
q = PosteriorFactorization(vars[:π], 
                           expand([[vars[pad(:m,k)], vars[pad(:w,k)]] for k=1:nr_clusters])...,
                           [vars[pad(:z,k)] for k=1:nr_samples],
                           [vars[pad(:Xc,k)] for k=1:nr_samples],
                           [vars[pad(:ξ,k)] for k=1:nr_samples],
                           ids=q_ids)

# generate the inference algorithm
algo = variationalAlgorithm(q)

# Generate source code
source_code = algorithmSourceCode(algo)

# Load algorithm
eval(Meta.parse(source_code))
;

## Calculate priors over means through K-means clustering and the EM-algorithm

In [7]:
# reshape Y
Yi = hcat(Y_samples...)

# approximate with log-power
Yi = collect(log.(abs2.(Yi[1:nr_freqs,:] + 1im*Yi[nr_freqs+1:end,:]))')

# perform clustering
g = GMM(nr_clusters, Yi, nIter=50, nInit=100, kind=:diag)
em!(g, Yi)
;

  Iters               objv        objv-change | affected 
-------------------------------------------------------------
      0       2.476037e+03
      1       1.600574e+03      -8.754635e+02 |        3
      2       1.593951e+03      -6.622835e+00 |        0
      3       1.593951e+03       0.000000e+00 |        0

┌ Info: Initializing GMM, 3 Gaussians diag covariance 5 dimensions using 200 data points
└ @ GaussianMixtures C:\Users\s151781\.julia\packages\GaussianMixtures\3jRIL\src\train.jl:78



K-means converged with 3 iterations (objv = 1593.9507847108434)


┌ Info: K-means with 200 data points using 3 iterations
│ 11.1 data points per parameter
└ @ GaussianMixtures C:\Users\s151781\.julia\packages\GaussianMixtures\3jRIL\src\train.jl:139


## Create data and marginals dictionary

In [8]:
# create data dictionary
data = Dict()

# specify input data and measurement noise
data[:Y] = Y_samples

# specify priors over class probabilities
data[:α_π] = 1.0*ones(nr_clusters)

# specify priors over clusters
for k = 1:nr_clusters
    data[pad(:μ_m,k)] = g.μ[k,:]
    data[pad(:nu_w,k)] = nr_freqs
    data[pad(:V_w,k)] = diagm(1 ./g.Σ[k,:]) / nr_freqs
end
;

In [9]:
# create marginals dictionary
marginals = Dict()

# specify marginals over class probabilities
marginals[:vars_π] = vague(ForneyLab.Dirichlet, nr_clusters)

# specify marginals over clusters
for k = 1:nr_clusters
    marginals[pad(:vars_m,k)] = ProbabilityDistribution(Multivariate, GaussianMeanPrecision, m=data[pad(:μ_m,k)], w=data[pad(:V_w,k)]*data[pad(:nu_w,k)])
    marginals[pad(:vars_w,k)] = ProbabilityDistribution(MatrixVariate, ForneyLab.Wishart, v=data[pad(:V_w,k)], nu=data[pad(:nu_w,k)])
end

# specify marginals over samples
for k = 1:nr_samples
    marginals[pad(:vars_z,k)] = vague(Categorical)
    marginals[pad(:vars_Xc,k)] = ProbabilityDistribution(Multivariate, ComplexNormal, μ=zeros(nr_freqs) .+ 0.0im, Γ=1e10*Ic(nr_freqs).+0.0im, C=mat(0.0+0.0im));
    marginals[pad(:vars_ξ,k)] = ProbabilityDistribution(Multivariate, GaussianMeanVariance, m=zeros(nr_freqs), v=Ic(nr_freqs))
end
; 

## Perform inference

In [10]:
# set number of iterations
nr_its = 10

# create progress bar
p = Progress(nr_its)

# perform iterations
for i = 1:nr_its
    
    # perform updates
    stepXc!(data, marginals)
    stepΞ!(data, marginals)
    stepZ!(data, marginals)
    stepΠ!(data, marginals) 
    for k = 1:nr_clusters
        getfield(Main, Symbol("stepM_"*string(k,pad=2)*"!"))(data, marginals)
        getfield(Main, Symbol("stepW_"*string(k,pad=2)*"!"))(data, marginals)
    end
    
    # update progress bar
    next!(p)
    
end

Progress: 100%|█████████████████████████████████████████| Time: 0:05:31


## Analyze inferred cluster assignments

In [11]:
z_inferred = [findmax(marginals[pad(:vars_z,k)].params[:p])[2] for k = 1:nr_samples];

In [12]:
function confusionmatrix(k::Integer, gts::Array{Int64,1}, preds::Array{Int64,1})
    # borrowed from MLBase
    n = length(gts)
    length(preds) == n || throw(DimensionMismatch("Inconsistent lengths."))
    R = zeros(Int, k, k)
    for i = 1:n
        @inbounds g = gts[i]
        @inbounds p = preds[i]
        R[g, p] += 1
    end
    return R
end

confusionmatrix(nr_clusters, cluster_id .- minimum(cluster_id) .+ 1, z_inferred .- minimum(z_inferred) .+ 1)

3×3 Array{Int64,2}:
 64   0   0
  0  70   0
  0   2  64

In [13]:
g.μ

3×5 Array{Float64,2}:
 0.047027  1.24658  2.50007   3.00019   4.59574
 1.33687   3.50789  5.50818   7.15009   9.69943
 2.66747   5.80304  8.4861   11.5997   14.4736 

In [14]:
for k = 1:nr_clusters
    println(ForneyLab.unsafeMean(marginals[pad(:vars_m,k)]))
end

[0.04876835551318326, 1.2448231055089947, 2.501001130462622, 3.0001092337285775, 4.609151413737056]
[1.3402234069963332, 3.5186433259400465, 5.500616731084405, 7.1588516151403825, 9.716459313299525]
[2.6822840279243287, 5.805194210205965, 8.50725161375487, 11.61623575331676, 14.46282083483107]
